# Cap to CL
This notebook seeks to organize court listener bulk data in a way similar to https://case.law


In [ ]:
!pip3 install lil-nocap

In [1]:
import lil_nocap as lnc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
# pandas option to show all columns
pd.set_option("display.max_columns", None)

## Get Files Needed

This assumes all the files have been [downloaded](https://com-courtlistener-storage.s3-us-west-2.amazonaws.com/list.html?prefix=bulk-data/) locally:
* courts
* dockets
* opinion_clusters
* opinions
* citation_map



## Pass file locations to NoCAP

In [4]:
path = "/Users/sabelosethumhlambi/Downloads/"

In [5]:
opinions_fn = f'{path}opinions-2022-12-31.csv'
courts_fn = f'{path}courts-2022-12-31.csv'
dockets_fn = f'{path}dockets-2022-12-31.csv'
opinion_clusters_fn = f'{path}opinion-clusters-2022-12-31.csv'
citation_fn = f'{path}citation-map-2022-12-31.csv'

## Initalize NoCAP

In [ ]:
nc = lnc.NoCap(opinions_fn, opinion_clusters_fn, courts_fn, dockets_fn, citation_fn)

Importing /Users/sabelosethumhlambi/Downloads/courts-2022-12-31.csv as a dataframe
File Size is : 0.0 GB
/Users/sabelosethumhlambi/Downloads/courts-2022-12-31.csv read in 0 minutes
Importing /Users/sabelosethumhlambi/Downloads/opinion-clusters-2022-12-31.csv as a dataframe
File Size is : 6.7 GB


/Users/sabelosethumhlambi/Library/Python/3.8/lib/python/site-packages/lil_nocap/__init__.py:33: DtypeWarning: Columns (10,17,18,19,20,21,22,23,24,25,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in pd.read_csv(filename, chunksize=max_rows, dtype=dtype,


/Users/sabelosethumhlambi/Downloads/opinion-clusters-2022-12-31.csv read in 2 minutes
Importing /Users/sabelosethumhlambi/Downloads/citation-map-2022-12-31.csv as a dataframe
File Size is : 0.79 GB
/Users/sabelosethumhlambi/Downloads/citation-map-2022-12-31.csv read in 0 minutes
Importing /Users/sabelosethumhlambi/Downloads/dockets-2022-12-31.csv as a dataframe
File Size is : 18.49 GB


/Users/sabelosethumhlambi/Library/Python/3.8/lib/python/site-packages/lil_nocap/__init__.py:33: DtypeWarning: Columns (7,22,34) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in pd.read_csv(filename, chunksize=max_rows, dtype=dtype,
/Users/sabelosethumhlambi/Library/Python/3.8/lib/python/site-packages/lil_nocap/__init__.py:33: DtypeWarning: Columns (7,20,22,31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in pd.read_csv(filename, chunksize=max_rows, dtype=dtype,
/Users/sabelosethumhlambi/Library/Python/3.8/lib/python/site-packages/lil_nocap/__init__.py:33: DtypeWarning: Columns (7,22,34) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in pd.read_csv(filename, chunksize=max_rows, dtype=dtype,
/Users/sabelosethumhlambi/Library/Python/3.8/lib/python/site-packages/lil_nocap/__init__.py:33: DtypeWarning: Columns (7,22,34) have mixed types. Specify dtype option on import or set low

## Process files

Generate JSON files with a default taxonomy

In [ ]:
nc.start()

## Analytics

In [695]:
mask = (
        df_opinions[('plain_text')].notna() |
        df_opinions[('html')].notna() |
        df_opinions[('html_lawbox')].notna() |
        df_opinions[('html_columbia')].notna() |
        df_opinions[('xml_harvard')].notna() |
        df_opinions[('html_anon_2020')].notna()

       )

In [696]:
texts = df_opinions[mask][[
        'plain_text',
        'html',
        'html_lawbox',
        'html_columbia',
        'xml_harvard',
        'html_anon_2020'
    
]]

In [697]:
labels = pd.Series(['plain_text',
        'html',
        'html_lawbox',
        'html_columbia',
        'xml_harvard',
        'html_anon_2020'])

In [699]:
amt = pd.Series([len(texts[label].dropna()) 
                    for label in labels])/len(texts)

In [700]:
df_opinions_chart = pd.DataFrame({'source':source, 'amount':amt})

In [60]:
df_opinions_chart